### 在迁移学习中，针对不同的层设置不同的学习率。对于backbone来说，已经是经过训练了，所以设置一个较小的学习率，对于后面的head设置一个较大的学习率，使得它更好的收敛。

In [2]:
#   在定义优化器的时候，用 list 将参数设置为不同的组，每个组( list 中的每个元素 )用字典表示，在字典中指明 参数组、该组的学习率
optimizer = optim.SGD([{'params': params_group_1, 'lr': 0.001},
                       {'params': params_group_2, 'lr': 0.0005}]) 

NameError: name 'optim' is not defined

In [ ]:
# 若分组未特别指定学习率或其他参数，可使用默认值
optimizer = optim.SGD([{'params': params_group_1},   # 未指定学习率的，使用默认学习率 0.001
                       {'params': params_group_2, 'lr': 0.0005}],
                      lr=0.001, momentum=0.9)   # 默认值

# 代码模板 1、将网络中的参数 按照模块进行分组
  按照模块分组，一般是对不同的 模块设置不同的学习率

In [3]:
import torchvision.models as models
import torch

model = models.resnet50(weights=None)
# module_names = [name for name, layer in model.named_children()]

layers = ['layer3', 'layer4', 'fc']

g = [], []
for name, param in model.named_parameters():
    if any([layer in name for layer in layers]):
        g[1].append(param) # 存储属于 layer3、layer4 或 fc 的参数。
    else:
        g[0].append(param) # 存储不需要特殊学习率的参数。

optimizer = torch.optim.SGD([{'params': g[0]},   # 未指定学习率的，使用默认学习率 0.0005
                             {'params': g[1], 'lr': 0.001}],
                            lr=0.0005, momentum=0.9)

print(optimizer.param_groups[0]['lr'])   # 0.0005
print(optimizer.param_groups[1]['lr'])   # 0.001

0.0005
0.001


<!-- 举一个简单的例子说明
为了更清楚地理解，我们来看一个具体的例子。

示例场景：
假设有一个层名称列表：layers = ['layer3', 'layer4', 'fc']。
我们要检查的参数名称是：name = 'layer3.0.conv1.weight'。
逐步计算：
执行列表推导式：
对于 layers 中的每个元素，检查它是否在 name 中：
'layer3' in 'layer3.0.conv1.weight' → 是的，返回 True（因为 'layer3' 是 name 的子字符串）。
'layer4' in 'layer3.0.conv1.weight' → 否，返回 False（name 中没有 'layer4'）。
'fc' in 'layer3.0.conv1.weight' → 否，返回 False（name 中没有 'fc'）。
结果是一个布尔值列表：[True, False, False]。
应用 any() 函数：
any([True, False, False]) 检查这个列表。
因为列表中至少有一个 True（来自 'layer3'），所以 any() 返回 True。
另一个场景：
如果 name = 'layer1.0.conv1.weight'：
'layer3' in 'layer1.0.conv1.weight' → False。
'layer4' in 'layer1.0.conv1.weight' → False。
'fc' in 'layer1.0.conv1.weight' → False。
布尔值列表为：[False, False, False]。
any([False, False, False]) 返回 False，因为没有一个层名称匹配。 -->

举一个简单的例子说明
为了更清楚地理解，我们来看一个具体的例子。

示例场景：
假设有一个层名称列表：layers = ['layer3', 'layer4', 'fc']。
我们要检查的参数名称是：name = 'layer3.0.conv1.weight'。
逐步计算：
执行列表推导式：
对于 layers 中的每个元素，检查它是否在 name 中：
'layer3' in 'layer3.0.conv1.weight' → 是的，返回 True（因为 'layer3' 是 name 的子字符串）。
'layer4' in 'layer3.0.conv1.weight' → 否，返回 False（name 中没有 'layer4'）。
'fc' in 'layer3.0.conv1.weight' → 否，返回 False（name 中没有 'fc'）。
结果是一个布尔值列表：[True, False, False]。
应用 any() 函数：
any([True, False, False]) 检查这个列表。
因为列表中至少有一个 True（来自 'layer3'），所以 any() 返回 True。

另一个场景：
如果 name = 'layer1.0.conv1.weight'：
'layer3' in 'layer1.0.conv1.weight' → False。
'layer4' in 'layer1.0.conv1.weight' → False。
'fc' in 'layer1.0.conv1.weight' → False。
布尔值列表为：[False, False, False]。
any([False, False, False]) 返回 False，因为没有一个层名称匹配。